# XGBost, LightGBM, and CatBoost

In [ ]:
# Load Libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import multiprocessing
import random
import io




from sklearn.preprocessing import scale
from sklearn.preprocessing import OneHotEncoder
import timeit

## Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving healthcare.csv to healthcare.csv


In [ ]:
X = pd.read_csv(io.BytesIO(uploaded['healthcare.csv']), sep = ";")
y = X['exitus']
X = X.drop(['exitus'], axis = 1)

### One-Hot Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categorical_vars = set(['severity', 'origin', 'tip_adm', 'tip_grd', 'date'])
numerical_vars = set(X.columns) - categorical_vars
categorical_vars = list(categorical_vars)
numerical_vars = list(numerical_vars)
ohe = OneHotEncoder(sparse_output = False)
ohe_fit = ohe.fit(X[categorical_vars])
X_ohe = pd.DataFrame(ohe.fit_transform(X[categorical_vars]))
X_ohe.columns = pd.DataFrame(ohe_fit.get_feature_names_out())
X = pd.concat((X_ohe, X[numerical_vars].reset_index()), axis=1)

In [ ]:
perc_values = [0.7, 0.15, 0.15];
# dimensiones de los conjuntos de train y test
n_train = int(X.shape[0] * perc_values[0])
n_val = int(X.shape[0] * perc_values[1])
n_test = int(X.shape[0] * perc_values[2])

# selección del conjunto de train
X_train = X.iloc[:n_train]
y_train = y.iloc[:n_train]

# selección del conjunto de validación
X_val = X.iloc[(n_train):(n_train+n_val)]
y_val = y.iloc[(n_train):(n_train+n_val)]

# selección del conjunto de test
X_test = X.iloc[(n_train+n_val):]
y_test = y.iloc[(n_train+n_val):]

## [2] XGBoost

1) Import model.

In [ ]:
from xgboost import XGBClassifier as model_constructor

2) Import metric.

In [ ]:
from sklearn.metrics import roc_auc_score as metric

3) Define model.

In [ ]:
model = model_constructor(random_state = 1,
          eval_metric = "auc",
          early_stopping_rounds = 10,
                          tree_method = 'hist')

4) Train model.

In [ ]:
import timeit
start = timeit.default_timer()
model.fit(X_train,
          np.array(y_train),
          eval_set=[(X_val, y_val)],
          verbose=True)
time = timeit.default_timer() - start

[0]	validation_0-auc:0.91698
[1]	validation_0-auc:0.91684
[2]	validation_0-auc:0.91877
[3]	validation_0-auc:0.92390
[4]	validation_0-auc:0.92573
[5]	validation_0-auc:0.92136
[6]	validation_0-auc:0.90169
[7]	validation_0-auc:0.90850
[8]	validation_0-auc:0.91820
[9]	validation_0-auc:0.92077
[10]	validation_0-auc:0.92101
[11]	validation_0-auc:0.92404
[12]	validation_0-auc:0.92377
[13]	validation_0-auc:0.92449


5) Predict.

In [ ]:
pred_train_p = model.predict_proba(X_train)
pred_val_p = model.predict_proba(X_val)
pred_test_p = model.predict_proba(X_test)

6) Evaluate.

In [ ]:
auc_train = metric(y_train, pred_train_p[:,1]);
auc_val = metric(y_val, pred_val_p[:,1]);
auc_test = metric(y_test, pred_test_p[:,1]);
results = pd.DataFrame()
results = results.append(pd.DataFrame(data={'model':['XGBoost'],'auc_train':[auc_train],'auc_val':[auc_val],'auc_test':[auc_test], 'time': [time]}, columns=['model',  'auc_train','auc_val', 'auc_test', 'time']), ignore_index=True)
results

<ipython-input-13-fcf004ffdd72>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(pd.DataFrame(data={'model':['XGBoost'],'auc_train':[auc_train],'auc_val':[auc_val],'auc_test':[auc_test], 'time': [time]}, columns=['model',  'auc_train','auc_val', 'auc_test', 'time']), ignore_index=True)


,model,auc_train,auc_val,auc_test,time
0,XGBoost,0.958681,0.925734,0.915126,1.010313



## [2] LightGBM


In [ ]:
%pip install lightgbm

In [ ]:
import lightgbm as lgb

The datasets must be **transformed into a specific format**.

In [ ]:
X_train.columns = ["var_" + str(x) for x in range(X.shape[1])]
X_val.columns = ["var_" + str(x) for x in range(X.shape[1])]
X_test.columns = ["var_" + str(x) for x in range(X.shape[1])]
train_data = lgb.Dataset(data = X_train, label = y_train)
validation_data = train_data.create_valid(data = X_val, label = y_val)

We train the model **(we do not use constructor + fit, but lgb.train)**.

In [ ]:
param = {'metric':'auc'}
start = timeit.default_timer()
model = lgb.train(param, train_data,
                  valid_sets=[validation_data],
                  callbacks=[lgb.early_stopping(stopping_rounds=10)])
time = timeit.default_timer() - start

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003050 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 907
[LightGBM] [Info] Number of data points in the train set: 22894, number of used features: 30
[LightGBM] [Info] Start training from score 0.038656
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[21]	valid_0's auc: 0.936216


5) Predict.

In [ ]:
pred_train_p = model.predict(X_train)
pred_val_p = model.predict(X_val)
pred_test_p = model.predict(X_test)

6) Evaluate.

In [ ]:
# Calcular métricas de evaluación
auc_train = metric(y_train, pred_train_p);
auc_val = metric(y_val, pred_val_p);
auc_test = metric(y_test, pred_test_p);
results = results.append(pd.DataFrame(data={'model':['LightGBM'],'auc_train':[auc_train],'auc_val':[auc_val],'auc_test':[auc_test], 'time': [time]}, columns=['model',  'auc_train','auc_val', 'auc_test', 'time']), ignore_index=True)
results

<ipython-input-19-1e6ca50b56bd>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(pd.DataFrame(data={'model':['LightGBM'],'auc_train':[auc_train],'auc_val':[auc_val],'auc_test':[auc_test], 'time': [time]}, columns=['model',  'auc_train','auc_val', 'auc_test', 'time']), ignore_index=True)


,model,auc_train,auc_val,auc_test,time
0,XGBoost,0.958681,0.925734,0.915126,1.010313
1,LightGBM,0.959469,0.936216,0.933622,0.170747



## [3] CatBoost

In [1]:
%pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.8 MB/s eta 0:00:00


1) Import model.

In [ ]:
from catboost import CatBoostClassifier as model_constructor

3) Define model.

In [ ]:
model = model_constructor(eval_metric='AUC', verbose=True)

4) Train model.

In [ ]:
start = timeit.default_timer()
model = model.fit(X_train, y_train,
                 eval_set=(X_val, y_val),
                  early_stopping_rounds = 10)
time = timeit.default_timer() - start

Learning rate set to 0.068669
0:	test: 0.8506362	best: 0.8506362 (0)	total: 57.9ms	remaining: 57.8s
1:	test: 0.9074548	best: 0.9074548 (1)	total: 66.5ms	remaining: 33.2s
2:	test: 0.9130314	best: 0.9130314 (2)	total: 74.3ms	remaining: 24.7s
3:	test: 0.9284293	best: 0.9284293 (3)	total: 82.8ms	remaining: 20.6s
4:	test: 0.9297097	best: 0.9297097 (4)	total: 92.4ms	remaining: 18.4s
5:	test: 0.9325922	best: 0.9325922 (5)	total: 101ms	remaining: 16.7s
6:	test: 0.9330743	best: 0.9330743 (6)	total: 109ms	remaining: 15.5s
7:	test: 0.9341399	best: 0.9341399 (7)	total: 118ms	remaining: 14.6s
8:	test: 0.9347349	best: 0.9347349 (8)	total: 126ms	remaining: 13.9s
9:	test: 0.9362803	best: 0.9362803 (9)	total: 134ms	remaining: 13.3s
10:	test: 0.9375132	best: 0.9375132 (10)	total: 142ms	remaining: 12.8s
11:	test: 0.9389498	best: 0.9389498 (11)	total: 150ms	remaining: 12.4s
12:	test: 0.9390203	best: 0.9390203 (12)	total: 161ms	remaining: 12.2s
13:	test: 0.9395161	best: 0.9395161 (13)	total: 168ms	remainin

5) Predict.

In [ ]:
pred_train_p = model.predict_proba(X_train)
pred_val_p = model.predict_proba(X_val)
pred_test_p = model.predict_proba(X_test)

6) Evaluate.

In [ ]:
# Calcular métricas de evaluación
auc_train = metric(y_train, pred_train_p[:,1]);
auc_val = metric(y_val, pred_val_p[:,1]);
auc_test = metric(y_test, pred_test_p[:,1]);
results = results.append(pd.DataFrame(data={'model':['CatBoost'],'auc_train':[auc_train],'auc_val':[auc_val],'auc_test':[auc_test], 'time': [time]}, columns=['model',  'auc_train','auc_val', 'auc_test', 'time']), ignore_index=True)
results

<ipython-input-25-f58209ce158c>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(pd.DataFrame(data={'model':['CatBoost'],'auc_train':[auc_train],'auc_val':[auc_val],'auc_test':[auc_test], 'time': [time]}, columns=['model',  'auc_train','auc_val', 'auc_test', 'time']), ignore_index=True)


,model,auc_train,auc_val,auc_test,time
0,XGBoost,0.958681,0.925734,0.915126,1.010313
1,LightGBM,0.959469,0.936216,0.933622,0.170747
2,CatBoost,0.955634,0.945425,0.939009,0.786112


Conclusions:
- Lightgbm is the fastest.
- Lightgbm requires some extra steps to run.
- Catboost has a very good performance with the default settings.
- A grid search should be performed for a more informative comparison on the final winner in terms of predictive tuning.